# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [79]:
# assign the collection to a variable
establishments = db['establishments']
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$convert': {'input': "$geocode.longitude", 'to': "decimal"}}}} ])
establishments.update_many({}, [ {'$set':{ "geocode.latitude" : {'$convert': {'input': "$geocode.latitude", 'to': "decimal"}}}} ])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [71]:
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])
establishments.update_many({}, [ {'$set':{ "RatingValue" : {'$toInt': '$RatingValue'}}} ])

UpdateResult({'n': 38786, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [72]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
h20 = establishments.count_documents(query)
# Display the first document in the results using pprint
pprint(establishments.find_one(query))

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65e94ebfdcdb2611902be587'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'longitude': None,
 'meta': 

In [74]:
# Convert the result to a Pandas DataFrame
h20 = list(establishments.find(query))
hygiene_df = pd.DataFrame(h20)

# Display the number of rows in the DataFrame
hygiene_df.shape[1]
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   _id                         41 non-null     object 
 1   FHRSID                      41 non-null     int64  
 2   ChangesByServerID           41 non-null     int64  
 3   LocalAuthorityBusinessID    41 non-null     object 
 4   BusinessName                41 non-null     object 
 5   BusinessType                41 non-null     object 
 6   BusinessTypeID              41 non-null     int64  
 7   AddressLine1                41 non-null     object 
 8   AddressLine2                41 non-null     object 
 9   AddressLine3                41 non-null     object 
 10  AddressLine4                41 non-null     object 
 11  PostCode                    41 non-null     object 
 12  Phone                       41 non-null     object 
 13  RatingValue                 41 non-nu

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [75]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex':'London'}, 'RatingValue': {'$gte': 4}}

# Use count_documents to display the number of documents in the result
establishments.count_documents(query)
# Display the first document in the results using pprint
pprint(establishments.find_one(query))

{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65e94ec0dcdb2611902bffaa'),
 'geocode': {'latitude': '51.369321', 'longitude': '0.508551'},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/621707',
    

In [76]:
# Convert the result to a Pandas DataFrame
london = list(establishments.find(query))
london_df = pd.DataFrame(london)
# Display the number of rows in the DataFrame
london_df.shape[1]
# Display the first 10 rows of the DataFrame
london_df.head(10)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links,longitude,latitude
0,65e94ec0dcdb2611902bffaa,621707,0,PI/000025307,Charlie's,Other catering premises,7841,Oak Apple Farm Building 103 Sheernes Docks,Sheppy Kent,,...,"{'Hygiene': 5, 'Structural': 10, 'ConfidenceIn...",FHRS,"{'longitude': '0.508551', 'latitude': '51.3693...",,4627.439468,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",None,None
1,65e94ec0dcdb2611902c02cd,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '0.249255999922752', 'latitude':...",,4627.873179,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
2,65e94ec0dcdb2611902c0e1d,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.561954', 'latitude': '51.5438...",,4631.968418,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
3,65e94ec0dcdb2611902c1c16,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '0.346336990594864', 'latitude':...",,4636.846754,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
4,65e94ec0dcdb2611902c1c19,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.346325', 'latitude': '51.4640...",,4636.847650,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
5,65e94ec0dcdb2611902c232b,294900,0,PI/000019066,Mv Valulla,Other catering premises,7841,Reeds River Cruises LtdKings ReachRiver Thames...,London,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.287555', 'latitude': '51.5040...",,4640.460834,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
6,65e94ec0dcdb2611902c4434,293756,0,PI/000002538,Tereza Joanne,Other catering premises,7841,Funcraft UK Ltd King George V Dock Woolwich Ma...,London,,...,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '0.069286', 'latitude': '51.5011...",,4648.301822,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
7,65e94ec0dcdb2611902c47fe,878523,0,PI/000029844,Brick Lane Brews,Restaurant/Cafe/Canteen,1,Air side London City Airport,London,,...,"{'Hygiene': 10, 'Structural': 5, 'ConfidenceIn...",FHRS,"{'longitude': '0.048006', 'latitude': '51.5037...",,4649.173485,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
8,65e94ec0dcdb2611902c4812,294606,0,PI/000016185,WH Smith,Retailers - other,4613,London City Airport,Silvertown London,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.0479655750095844', 'latitude'...",,4649.188850,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
9,65e94ec0dcdb2611902c4814,293772,0,PI/000002585,The Nuance Group (UK) Limited,Retailers - other,4613,Duty Free Shop Passenger Terminal,London City AirportRoyal DocksLondon,,...,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.047966', 'latitude'

In [77]:
pprint(establishments.find_one({'BusinessName': 'Penang Flavours'}, {'geocode.longitude': 1}))

{'_id': ObjectId('65e94f2df172605a03c61716'),
 'geocode': {'longitude': '0.08384000'}}


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [82]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
         'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search},
         'RatingValue': 5}
sort = [('scores.Hygiene', -1)]
limit = 5

results = list(establishments.find(query).sort(sort).limit(limit))

In [84]:
# Convert result to Pandas DataFrame
top5_df = pd.DataFrame(results)
top5_df

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links,longitude,latitude
0,65e94ec0dcdb2611902c3e92,695287,0,PI/000182135,Lucky Food & Wine,Retailers - other,4613,101 Plumstead High Street,,Plumstead,...,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0910104, 'latitude': 51.4878934}",,4647.024793,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
1,65e94ec0dcdb2611902c3ec5,695033,0,PI/000172986,Everest Stores Ltd,Retailers - other,4613,104 Plumstead High Street,,Plumstead,...,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0903232, 'latitude': 51.4876718}",,4647.041728,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
2,65e94ec0dcdb2611902c3ea6,695566,0,PI/000188497,Premier Express,Retailers - other,4613,102 Plumstead High Street,,Plumstead,...,"{'Hygiene': 5, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0904552, 'latitude': 51.4877234}",,4647.038807,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
3,65e94ec0dcdb2611902c3e78,694606,0,PI/000116584,Fineway Cash & Carry,Retailers - other,4613,112 Plumstead High Street,,Plumstead,...,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0909382, 'latitude': 51.4876089}",,4647.016995,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None
4,65e94ec0dcdb2611902c3d98,1069652,0,PI/000206841,TIWA N TIWA African Restaurant Ltd,Restaurant/Cafe/Canteen,1,152 Plumstead High Street,,Plumstead,...,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0927429, 'latitude': 51.4870351}",,4646.930146,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f...",None,None


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [102]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match': {'scores.Hygiene': 0}}
# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}
# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': {'count': -1}}
# Print the number of documents in the result
pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))
# Print the first 10 results
pprint(results[0:10])

[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [108]:
# Convert the result to a Pandas DataFrame
group_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
group_df.shape[0]
# Display the first 10 rows of the DataFrame
group_df.head(10)

,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
